<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/4/30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import dash
from dash import html, dcc, Input, Output, State
import time
import threading
import pandas as pd
import plotly.express as px
import numpy as np
import random

# 初始化 Dash App
app = dash.Dash(__name__)
server = app.server

# 所有報告步驟名稱
step_names = [
    "撈取資料",
    "確認與篩選資料",
    "資料預處理",
    "進行 Rule Check",
    "繪圖",
    "上拋資料"
]

# 用來儲存流程進度的全域變數
progress = {
    'current_index': -1,                      # 當前執行的步驟 index
    'step_times': [None] * len(step_names),   # 每步驟所花費的秒數
    'step_status': ['pending'] * len(step_names),  # 步驟狀態: pending, running, success, error
    'fig': None,                              # 儲存圖表結果
    'error': False                            # 是否有錯誤發生
}

# 模擬報告處理流程（會在背景執行）
def run_report_process():
    df = None
    for i in range(len(step_names)):
        if progress['error']:
            break  # 若出錯則中止流程

        progress['current_index'] = i
        progress['step_status'][i] = 'running'
        step_start = time.time()

        # 模擬每步驟花費時間（撈取與繪圖可以拉長）
        if i in [0, 4]:  # 撈取資料與繪圖步驟
            time.sleep(random.randint(1, 3))  # 模擬長處理（實際可放大為10~20分鐘）
        else:
            time.sleep(1)

        # 模擬第 4 步（rule check）有機率發生錯誤
        if i == 3 and random.random() < 0.1:
            progress['step_status'][i] = 'error'
            progress['error'] = True
            break

        # 繪圖步驟：建立圖表
        if i == 4:
            df = pd.DataFrame({
                'player_id': range(1, 101),
                'profit': np.random.normal(1000, 300, 100),
                'game_type': np.random.choice(['A', 'B', 'Cash', 'MTT'], 100)
            })
            fig = px.histogram(df, x='profit', color='game_type', nbins=20, title="玩家盈利分佈")
            progress['fig'] = fig

        # 儲存步驟完成時間與狀態
        step_end = time.time()
        progress['step_times'][i] = round(step_end - step_start, 2)
        progress['step_status'][i] = 'success'

# App 畫面 layout
app.layout = html.Div([
    html.H2("報告產出流程 - 步驟狀態控制 + 效能優化"),

    html.Button("開始產出報告", id="run-btn", n_clicks=0),  # 啟動流程按鈕

    html.Div(id="step-list", style={"marginTop": "20px", "fontSize": "16px"}),  # 步驟狀態顯示區
    html.Div(id="progress-label", style={"marginTop": "20px", "fontSize": "16px"}),  # 狀態文字區

    dcc.Interval(id="interval", interval=2000, n_intervals=0, disabled=True),  # 每2秒檢查進度變化
    dcc.Loading(html.Div(id="chart-output")),  # 圖表結果呈現區，Loading 提供 loading animation

    html.Div(id="to-comment-btn", style={"marginTop": "30px"}),  # 完成後出現的按鈕

    dcc.Store(id="last-step-store", data=-1)  # 儲存前一次步驟 index，用來減少不必要的畫面刷新
])

# 當按下「開始報告」按鈕時，初始化流程狀態與啟動背景執行
@app.callback(
    Output("interval", "disabled"),
    Output("last-step-store", "data"),
    Input("run-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_report(n_clicks):
    # 重設所有流程參數
    progress['current_index'] = -1
    progress['step_times'] = [None] * len(step_names)
    progress['step_status'] = ['pending'] * len(step_names)
    progress['fig'] = None
    progress['error'] = False

    # 背景執行流程
    thread = threading.Thread(target=run_report_process)
    thread.start()

    return False, -1  # 啟用 interval、自動刷新開啟

# 每2秒檢查一次流程進度，只在變化時才更新畫面（效能優化）
@app.callback(
    Output("step-list", "children"),
    Output("progress-label", "children"),
    Output("chart-output", "children"),
    Output("to-comment-btn", "children"),
    Output("last-step-store", "data"),
    Input("interval", "n_intervals"),
    State("last-step-store", "data"),
    prevent_initial_call=True
)
def update_ui(n, last_index):
    current = progress['current_index']

    # (!) 若進度未改變，不更新畫面（節省資源）
    if current == last_index:
        raise dash.exceptions.PreventUpdate

    step_status = progress['step_status']
    step_times = progress['step_times']
    steps_total = len(step_names)
    chart = dcc.Graph(figure=progress['fig']) if progress['fig'] else ""
    btn = ""

    # 組合步驟清單顯示，附上狀態圖示與耗時
    step_elements = []
    for i, step in enumerate(step_names):
        status = step_status[i]
        if status == 'pending':
            color, prefix = 'gray', "□"
        elif status == 'running':
            color, prefix = 'orange', "▶"
        elif status == 'success':
            color, prefix = 'green', "✔"
        elif status == 'error':
            color, prefix = 'red', "❌"
        else:
            color, prefix = 'gray', "□"

        time_spent = f" ({step_times[i]}秒)" if step_times[i] is not None else ""
        step_elements.append(html.Div(f"{prefix} {step}{time_spent}", style={"color": color, "marginBottom": "5px"}))

    # 判斷標籤與是否顯示按鈕
    if progress['error']:
        label = f"流程中斷：{step_names[current]} 發生錯誤"
    elif current >= steps_total - 1 and all(s == 'success' for s in step_status):
        label = "✅ 已完成所有步驟！"
        btn = html.Button("TO COMMENT", id="to-comment", n_clicks=0, style={"fontSize": "16px"})
    else:
        label = f"目前進度：{current + 1}/{steps_total}"

    return step_elements, label, chart, btn, current

# 啟動 App
if __name__ == "__main__":
    app.run(debug=True)
